In [1]:
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
from sklearn.model_selection import train_test_split 
from torchmetrics import Accuracy , HingeLoss

In [2]:
train_path = 'https://raw.githubusercontent.com/FabioNicotra/fabionicotra.github.io/main/02_MobileClassification/train.csv'
test_path = 'https://raw.githubusercontent.com/FabioNicotra/fabionicotra.github.io/main/02_MobileClassification/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [3]:
train_df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
x = train_df.drop('price_range' , axis=1)
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [5]:
y = train_df['price_range']
y

0       1
1       2
2       2
3       2
4       1
       ..
1995    0
1996    2
1997    3
1998    0
1999    3
Name: price_range, Length: 2000, dtype: int64

In [6]:
x_train , x_validation , y_train , y_validation = train_test_split(x , y , train_size = 0.7 , random_state = 101)

In [7]:
x_train.shape

(1400, 20)

In [8]:
x_validation.shape

(600, 20)

In [9]:
x_train = torch.FloatTensor(x_train.values)
x_validation = torch.FloatTensor(x_validation.values)

y_train = torch.LongTensor(y_train.values)
y_validation = torch.LongTensor(y_validation.values)

In [10]:
mu = x_train.mean(dim = 0)
std = x_train.std(dim = 0)

In [11]:
x_train = (x_train - mu) / std
x_validation = (x_validation - mu) / std

In [12]:
train_dataset = TensorDataset(x_train , y_train)
valid_dataset = TensorDataset(x_validation , y_validation)

train_dataset.tensors

(tensor([[-0.2050,  1.0155,  1.4617,  ...,  0.5683, -1.0068,  0.9897],
         [-1.3516, -0.9841,  0.4723,  ..., -1.7582, -1.0068,  0.9897],
         [-1.6766,  1.0155,  1.5854,  ...,  0.5683, -1.0068, -1.0097],
         ...,
         [ 0.6821,  1.0155, -0.0223,  ...,  0.5683, -1.0068, -1.0097],
         [ 0.8220, -0.9841, -1.2591,  ...,  0.5683, -1.0068,  0.9897],
         [ 0.4022, -0.9841, -0.3934,  ...,  0.5683,  0.9925,  0.9897]]),
 tensor([2, 2, 2,  ..., 0, 2, 0]))

In [13]:
train_loader = DataLoader(train_dataset ,100 , True )
valid_loader = DataLoader(valid_dataset ,200 , False )
print(len(train_loader))

14


In [62]:
num_featur = 20
num_classes=4
h1 = 64
h2 = 32

model = nn.Sequential(
    nn.Linear(num_featur , h1),
    nn.ReLU(),
    nn.Linear(h1 , h2),
    nn.ReLU(),
    nn.Linear(h2 , num_classes)
)

📌 Loss

In [63]:
 #loss_fn = HingeLoss(task="multiclass" , num_classes = 4)

In [64]:
#loss_fn = nn.NLLLoss()

In [65]:
#تعریف کراس انتروپی از طریق nnloss
"""def loss_fn (outputs , targets):
    outputs = F.log_softmax(outputs , dim = 1)
    ## فرمولش کراس انتروپی اینه :  loss = - yt log(yp)
    return F.nll_loss(outputs , targets)
    """

'def loss_fn (outputs , targets):\n    outputs = F.log_softmax(outputs , dim = 1)\n    ## فرمولش کراس انتروپی اینه :  loss = - yt log(yp)\n    return F.nll_loss(outputs , targets)\n    '

In [66]:
#defin focal_loss
#فرمول فوکال لاس اینه : yp = softmax(output) ----->aipha (1-yp)**gama * log(yp)
def loss_fn (outputs , targets , alpha =0.25 , gamma = 2):
    weights = torch.ones( outputs.shape[1] ) * alpha
    prob = F.softmax(outputs , dim = 1)
    outputs = (1-prob)**gamma * torch.log(prob)
    #alpha is defines as weights and instead of pultypluying to output we git it as a weigth which do the same
    return F.nll_loss(outputs , targets , weights)


📌 optimizer 

In [67]:
#optimizer = optim.SGD(model.parameters() , lr =0.01)
#optimizer = optim.SGD(model.parameters() , lr =0.01 , momentum= 0.9 )
#optimizer = optim.SGD(model.parameters() , lr =0.01 , momentum= 0.9 , nesterov=True)
#optimizer = optim.RMSprop(model.parameters() , lr=0.01 , alpha = 0.99 )
optimizer = optim.RMSprop(model.parameters() , lr=0.0001 )

In [68]:
class AverageMeter(object):
    #computes and store the average and current value  
    def __init__(self):
        self.reset()
    def reset(self):
        self.val=0
        self.avg=0
        self.sum=0
        self.count=0
    def update(self , val , n=1):
        self.val=val
        self.sum += val*n
        self.count +=n
        self.avg = self.sum /self.count

In [69]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [70]:
num_epoch  = 200
loss_train_list = []
loss_valid_list = []
acc_train_list = []
acc_valid_list =[]

for epoch in range (num_epoch):
    for i , (inputs , targets) in enumerate(train_loader):
        acc_train = Accuracy(task="multiclass", num_classes=4).to(device)
        loss_train = AverageMeter()
        inputs = inputs.to(device)
        targets = targets.to(device)
        yp = model(inputs)        
        loss = loss_fn(yp , targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_train.update(loss.item())
        acc_train(yp , targets)
    
    with torch.no_grad():
        for i , (inputs,targets) in enumerate(valid_loader):
            acc_valid = Accuracy(task="multiclass", num_classes=4).to(device)
            loss_valid = AverageMeter()
            inputs = inputs.to(device)
            targets = targets.to(device)       
            yp = model(inputs)
            loss = loss_fn( yp , targets )
            loss_valid.update(loss.item())
            acc_valid(yp , targets)

                
    loss_train_list.append(loss_train.avg)
    loss_valid_list.append(loss_valid.avg)

    acc_train_list.append(acc_train.compute())
    acc_valid_list.append(acc_valid.compute())

    if (epoch % 10 ==0):
        print(f'epoch = {epoch}')
        print(f'Train:      loss = {loss_train.avg:.4} , accuracy = {acc_train.compute():.4}')  
        print(f'Valid:      loss = {loss_valid.avg:.4} , accuracy = {acc_valid.compute():.4}\n')


epoch = 0
Train:      loss = 0.7652 , accuracy = 0.38
Valid:      loss = 0.7859 , accuracy = 0.235

epoch = 10
Train:      loss = 0.6991 , accuracy = 0.59
Valid:      loss = 0.7145 , accuracy = 0.44

epoch = 20
Train:      loss = 0.6017 , accuracy = 0.68
Valid:      loss = 0.6183 , accuracy = 0.54

epoch = 30
Train:      loss = 0.4792 , accuracy = 0.72
Valid:      loss = 0.5182 , accuracy = 0.645

epoch = 40
Train:      loss = 0.3976 , accuracy = 0.74
Valid:      loss = 0.4238 , accuracy = 0.7

epoch = 50
Train:      loss = 0.3147 , accuracy = 0.82
Valid:      loss = 0.3434 , accuracy = 0.795

epoch = 60
Train:      loss = 0.2252 , accuracy = 0.88
Valid:      loss = 0.2769 , accuracy = 0.815

epoch = 70
Train:      loss = 0.1945 , accuracy = 0.9
Valid:      loss = 0.2236 , accuracy = 0.865

epoch = 80
Train:      loss = 0.1477 , accuracy = 0.95
Valid:      loss = 0.1805 , accuracy = 0.88

epoch = 90
Train:      loss = 0.124 , accuracy = 0.9
Valid:      loss = 0.1472 , accuracy = 0.91



In [44]:
optim_name = 'sgd_nest'
torch.save(torch.tensor(acc_train_list),f'{optim_name}-acc-train.pt')
torch.save(torch.tensor(acc_valid_list),f'{optim_name}-acc-valid.pt')

torch.save(torch.tensor(loss_train_list),f'{optim_name}-loss-train.pt')
torch.save(torch.tensor(loss_valid_list),f'{optim_name}-loss-valid.pt')